In [1]:
import torch

In [2]:
from model.AttributeEncoder import MultiLevelAttEncoder

In [3]:
X_t = torch.zeros([1, 3, 256, 256])

In [4]:
input_channels = [3, 32, 64, 128, 256, 512, 1024]
output_channels = [32, 64, 128, 256, 512, 1024, 1024]

att_enc = MultiLevelAttEncoder(input_channels, output_channels)

In [13]:

z_attributes = att_enc(X_t)

In [14]:
[*map(lambda x: x.shape, z_attributes)]

[torch.Size([1, 1024, 2, 2]),
 torch.Size([1, 2048, 4, 4]),
 torch.Size([1, 1024, 8, 8]),
 torch.Size([1, 512, 16, 16]),
 torch.Size([1, 256, 32, 32]),
 torch.Size([1, 128, 64, 64]),
 torch.Size([1, 64, 128, 128]),
 torch.Size([1, 64, 256, 256])]

In [16]:
z_id = torch.zeros([1, 256, 1, 1])

In [19]:
h_0 = torch.nn.ConvTranspose2d(256, 1024, [2, 2], 1, 0)(z_id)

torch.Size([1, 1024, 2, 2])

In [21]:
z_att_0 = z_attributes[0]

In [23]:
h_0.shape, z_att_0.shape, z_id.shape

(torch.Size([1, 1024, 2, 2]),
 torch.Size([1, 1024, 2, 2]),
 torch.Size([1, 256, 1, 1]))

In [24]:
h_1 = torch.zeros([1, 1024, 2, 2])

In [ ]:
channel_in, channel_out =

In [78]:
next_h = aad([h_0, z_id, z_att_0])

In [79]:
next_h.shape

torch.Size([1, 1024, 2, 2])

## AAD

batch_norm = torch.nn.BatchNorm2d(1024)
h_0_hat = batch_norm(h_0)


#adaptation mask M

conv_mask = torch.nn.Conv2d(1024, 1024, [2,2])
M_hat = conv_mask(h_0_hat)
M = torch.nn.Sigmoid()(M_hat)

# identity integration
gamma_fc = torch.nn.Linear(256, 1024)
gamma_id = gamma_fc(z_id.squeeze(-1).squeeze(-1))

beta_fc = torch.nn.Linear(256, 1024)
beta_id = beta_fc(z_id.squeeze(-1).squeeze(-1))

z_att_0.shape

#attributes integration
gamma_conv = torch.nn.Conv2d(1024, 1024, [2, 2])
gamma_att = gamma_conv(z_att_0)

beta_conv = torch.nn.Conv2d(1024, 1024, [2, 2])
beta_att = beta_conv(z_att_0)

I_mul = torch.mul(gamma_id.unsqueeze(-1).unsqueeze(-1), h_0_hat)
I = I_mul + beta_id.unsqueeze(-1).unsqueeze(-1)

A_mul = torch.mul(gamma_att, h_0_hat)
A = A_mul + beta_att

next_h = torch.mul((1 - M), A) + torch.mul(M, I)

In [85]:
k = 0

In [86]:
h_0.shape, z_att_0.shape, z_id.shape

(torch.Size([1, 1024, 2, 2]),
 torch.Size([1, 1024, 2, 2]),
 torch.Size([1, 256, 1, 1]))

In [81]:
input_channels = output_channels = 1024

z_input_channels=[256, 1024]

In [103]:
class DefaultResBLK(torch.nn.Module):
    def __init__(self, input_channels, output_channels, z_input_channels):
        super(DefaultResBLK, self).__init__()
        self.AAD = AADUnit(h_input_channels=input_channels,
                z_input_channels=z_input_channels,
                output_channels=input_channels ####
                       )
        self.conv = torch.nn.Conv2d(input_channels, output_channels, [3, 3], 1, 1)
        self.relu_op = torch.nn.ReLU()
    def forward(self, inputs):
        hidden, z_identity, z_attributes = inputs
        hidden = self.AAD([hidden, z_identity, z_attributes])
        hidden_r = self.relu_op(hidden)
        next_hidden = self.conv(hidden_r)
        
        return next_hidden

In [104]:
res_blk_0 = DefaultResBLK(input_channels=1024, output_channels=512, z_input_channels=z_input_channels)

In [105]:
res_blk_1 = DefaultResBLK(input_channels=1024, output_channels=512, z_input_channels=z_input_channels)

In [109]:
z_att_0.shape

torch.Size([1, 1024, 2, 2])

In [110]:
out = res_blk_0([h_0, z_id, z_att_0])

In [111]:
out.shape

torch.Size([1, 512, 2, 2])

In [88]:

h_inter = aad_0([h_0, z_id, z_att_0])# hidden, z_identity, z_attributes

relu_0 = torch.nn.ReLU()(h_inter)

h_inter = conv_0(relu_0)

In [174]:
class AADResBLK(torch.nn.Module):
    def __init__(self, input_channels, output_channels, z_input_channels):
        super(AADResBLK, self).__init__()
        self.additional_block = True if input_channels != output_channels else False
        
        if self.additional_block:
            
            self.res_blk_0 = DefaultResBLK(input_channels=input_channels, output_channels=input_channels,
                                                           z_input_channels=z_input_channels)
            self.res_blk_1 = DefaultResBLK(input_channels=input_channels, output_channels=output_channels,
                                                           z_input_channels=z_input_channels)
            self.res_blk_2 = DefaultResBLK(input_channels=input_channels, output_channels=output_channels,
                                                           z_input_channels=z_input_channels)
        else:
            self.res_blk_0 = DefaultResBLK(input_channels=input_channels, output_channels=output_channels,
                                                           z_input_channels=z_input_channels)
            self.res_blk_1 = DefaultResBLK(input_channels=input_channels, output_channels=output_channels,
                                                           z_input_channels=z_input_channels)
            
    def forward(self, inputs):
        hidden_init, z_identity, z_attributes = inputs
        if self.additional_block:
            
            hidden_0 = self.res_blk_0([hidden_init, z_identity, z_attributes])
            hidden_1 = self.res_blk_1([hidden_0, z_identity, z_attributes])
            
            hidden_2 = self.res_blk_2([hidden_init, z_identity, z_attributes])
            
            return hidden_1 + hidden_2
        else:
            hidden_0 = self.res_blk_0([hidden_init, z_identity, z_attributes])
            hidden_1 = self.res_blk_1([hidden_0, z_identity, z_attributes])
            return hidden_init + hidden_1
            
            
        
        

In [135]:
True if input_channels != output_channels else False

False

In [ ]:
torch.nn.Conv2d()

In [136]:
[*map(lambda x: x.shape, z_attributes)]

[torch.Size([1, 1024, 2, 2]),
 torch.Size([1, 2048, 4, 4]),
 torch.Size([1, 1024, 8, 8]),
 torch.Size([1, 512, 16, 16]),
 torch.Size([1, 256, 32, 32]),
 torch.Size([1, 128, 64, 64]),
 torch.Size([1, 64, 128, 128]),
 torch.Size([1, 64, 256, 256])]

In [137]:
aad_config = {
    0:{'input_dim': 1024,
      'output_dim': 1024,
      'z_input_channels': [256, 1024]
          },
    1:{'input_dim': 1024,
      'output_dim': 1024,
      'z_input_channels': [256, 2048]
          },
    2:{'input_dim': 1024,
      'output_dim': 1024,
      'z_input_channels': [256, 1024]
          },
    3:{'input_dim': 1024,
      'output_dim': 512,
      'z_input_channels': [256, 512]
          },
    4:{'input_dim': 512,
      'output_dim': 256,
      'z_input_channels': [256, 256]
          },
    5:{'input_dim': 256,
      'output_dim': 128,
      'z_input_channels': [256, 128]
          },
    6:{'input_dim': 128,
      'output_dim': 64,
      'z_input_channels': [256, 64]
          },
    7:{'input_dim': 64,
      'output_dim': 3,
      'z_input_channels': [256, 64]
          },
    }

In [404]:
class AADGenerator(torch.nn.Module):
    def __init__(self, aad_config):
        super(AADGenerator, self).__init__()
        self.layers = []
        self.h_init_layer = torch.nn.ConvTranspose2d(256, 1024, [2, 2], 1, 0)
        for params in aad_config.values():
            layer = AADResBLK(params['input_dim'], params['output_dim'], params['z_input_channels'])
            self.layers += [layer]
    def forward(self, inputs):
        z_attributes_dict, z_id = inputs
        h_init = self.h_init_layer(z_id)
        #hidden_init, z_identity, z_attributes = inputs
        hidden = h_init
        for k in range(8):
            resblk_inputs = [hidden, z_id, z_attributes_dict[k]]
            print(k)
            hidden = self.layers[k](resblk_inputs)
            print(hidden.shape , "before upsampling")
            hidden = torch.nn.UpsamplingBilinear2d(scale_factor=2)(hidden) ###!
            print(hidden.shape , "After upsampling")
        return hidden

In [405]:

z_attributes = att_enc(X_t)

In [406]:
z_id = torch.zeros([1, 256, 1, 1])

z_attributes_dict = {k:z_attributes[k] for k in range(8)}

In [407]:
aad_generator = AADGenerator(aad_config)

In [408]:
z_attributes_dict, z_id = generator_inputs

aad_generator(generator_inputs)

[*map(lambda x: x.shape, z_attributes_dict.values())]

In [246]:
k = 1

In [247]:
h_init = aad_generator.h_init_layer(z_id)

In [256]:
hidden = torch.nn.UpsamplingBilinear2d(scale_factor=2)(h_init)

In [249]:
z_att_1 = z_attributes_dict[k]

In [250]:
z_att_1.shape

torch.Size([1, 2048, 4, 4])

In [251]:
aad_config[k]

{'input_dim': 1024, 'output_dim': 1024, 'z_input_channels': [256, 2048]}

In [281]:
torch.nn.Conv2d(3, 3, [2,2], padding='same')

Conv2d(3, 3, kernel_size=[2, 2], stride=(1, 1), padding=same)

In [348]:
from functools import reduce
from operator import __add__

In [354]:
kernel_sizes = (2,)

In [355]:
conv_padding = reduce(__add__, 
    [(k // 2 + (k - 2 * (k // 2)) - 1, k // 2) for k in kernel_sizes[::-1]])

In [356]:
conv_padding

(0, 1)

In [403]:
class AADUnit(torch.nn.Module):
    
    def __init__(self, h_input_channels=1024, z_input_channels=[256, 1024], output_channels=1024):
        super(AADUnit, self).__init__()
        z_id_input_ch, z_att_input_ch = z_input_channels
        self.batch_norm = torch.nn.BatchNorm2d(output_channels)
        #adaptation mask M
        self.conv_mask = torch.nn.Conv2d(h_input_channels, output_channels, [3, 3])
        #identity integration
        self.gamma_fc = torch.nn.Linear(z_id_input_ch, output_channels)
        self.beta_fc = torch.nn.Linear(z_id_input_ch, output_channels)
        #attributes integration
        self.gamma_conv = torch.nn.Conv2d(z_att_input_ch, output_channels, [3, 3], stride=1, padding=1)
        self.beta_conv = torch.nn.Conv2d(z_att_input_ch, output_channels, [3, 3], stride=1, padding=1)
        
    
    def forward(self, inputs):
        hidden, z_identity, z_attributes = inputs
        
        hidden_norm = self.batch_norm(hidden)
        
        #adaptation mask
        M = self.conv_mask(hidden_norm)
        M = torch.nn.Sigmoid()(M)
        
        #identity integration
        gamma_id = self.gamma_fc(z_identity.squeeze(-1).squeeze(-1))
        beta_id = self.beta_fc(z_id.squeeze(-1).squeeze(-1))
        
        #attributes integration
        gamma_att = self.gamma_conv(z_attributes)
        beta_att = self.beta_conv(z_attributes)
        
        "(5) https://arxiv.org/pdf/1912.13457.pdf"
        I_mul = torch.mul(gamma_id.unsqueeze(-1).unsqueeze(-1), hidden_norm)
        I = I_mul + beta_id.unsqueeze(-1).unsqueeze(-1)

        A_mul = torch.mul(gamma_att, hidden_norm)
        A = A_mul + beta_att

        next_h = torch.mul((1 - M), A) + torch.mul(M, I)
        return next_h

In [390]:
layer = AADUnit(h_input_channels=aad_config[k]['input_dim'],
       z_input_channels=aad_config[k]['z_input_channels'],
       output_channels=aad_config[k]['output_dim'] )

In [391]:
hidden, z_identity, z_att = hidden, z_id, z_attributes_dict[k]

In [392]:
hidden_norm = layer.batch_norm(hidden)

M = layer.conv_mask(hidden_norm)
M = torch.nn.Sigmoid()(M)

In [393]:
hidden_norm.shape

torch.Size([1, 1024, 4, 4])

In [394]:
M.shape

torch.Size([1, 1024, 2, 2])

In [395]:
gamma_id = layer.gamma_fc(z_identity.squeeze(-1).squeeze(-1))
beta_id = layer.beta_fc(z_id.squeeze(-1).squeeze(-1))


In [396]:
gamma_id.shape

torch.Size([1, 1024])

In [397]:
gamma_att = layer.gamma_conv(z_att)
beta_att = layer.beta_conv(z_att)

In [398]:
gamma_att.shape

torch.Size([1, 1024, 4, 4])

In [399]:
I_mul = torch.mul(gamma_id.unsqueeze(-1).unsqueeze(-1), hidden_norm)
I = I_mul + beta_id.unsqueeze(-1).unsqueeze(-1)

I_mul.shape

torch.Size([1, 1024, 4, 4])

In [400]:
gamma_att.shape, hidden_norm.shape

(torch.Size([1, 1024, 4, 4]), torch.Size([1, 1024, 4, 4]))

In [401]:
A_mul = torch.mul(gamma_att, hidden_norm)

In [205]:
hidden.shape, z_id.shape, z_attributes_dict[k].shape

(torch.Size([1, 1024, 2, 2]),
 torch.Size([1, 256, 1, 1]),
 torch.Size([1, 2048, 4, 4]))

In [402]:
for k in range(8):
    resblk_inputs = [hidden, z_id, z_attributes_dict[k]]
    print(k)
    hidden = aad_generator.layers[k](resblk_inputs)

0


RuntimeError: The size of tensor a (3) must match the size of tensor b (4) at non-singleton dimension 3

In [185]:
aad_generator(generator_inputs)

0
1


RuntimeError: The size of tensor a (3) must match the size of tensor b (4) at non-singleton dimension 3

In [ ]:
torch.nn.UpsamplingBilinear2d()